In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.datasets import fetch_covtype

In [2]:
%run 'common.ipynb'

In [3]:
# Function to evaluate and print model performance
def evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Print model evaluation metrics
    evaluate(y_test, y_pred, f"\nModel: {model.__class__.__name__}")

In [9]:
NORMAL_TARGET = 2
TARGET_COLUMN = 'Cover_Type'

NUMERICAL_COLUMNS = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 
                     'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 
                     'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 
                     'Horizontal_Distance_To_Fire_Points']

# Dictionary to store models and their names
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Support Vector Machine (SVM)": SVC()
}

In [10]:
# Load the Cover Type dataset
data = fetch_covtype(as_frame=True)
X_data = data['data']
y_data = data['target']

all_df = pd.concat([X_data, y_data], axis=1)
all_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Cover_Type
0,2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,6225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
2,2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,6211.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,2595.0,45.0,2.0,153.0,-1.0,391.0,220.0,234.0,150.0,6172.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5


In [11]:
# Get X and y from all_df
X_df = all_df.drop(columns=[TARGET_COLUMN])
y_df = all_df[TARGET_COLUMN]

# Split the data into training and testing sets (80% train, 20% test)
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_df, y_df, test_size=0.2, random_state=42)
print(X_train_df.shape, X_test_df.shape, y_train_df.shape, y_test_df.shape)

(464809, 54) (116203, 54) (464809,) (116203,)


In [12]:
# Scale the features (required for some models, especially SVM)
scaler = StandardScaler()

# columns_to_scale = all_df.columns.get_indexer(NUMERICAL_COLUMNS)
# print('columns_to_scale', columns_to_scale)

# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

X_train_scaled_df = X_train_df.copy()
X_train_scaled_df[NUMERICAL_COLUMNS] = scaler.fit_transform(X_train_scaled_df[NUMERICAL_COLUMNS])

X_test_scaled_df = X_test_df.copy()
X_test_scaled_df[NUMERICAL_COLUMNS] = scaler.transform(X_test_scaled_df[NUMERICAL_COLUMNS])

In [ ]:
# Loop through models and evaluate each one
for model_name, model in models.items():
    # For SVM and Logistic Regression, use scaled data
    if model_name in ["Logistic Regression", "Support Vector Machine (SVM)"]:
        evaluate_model(model, X_train_scaled_df, y_train_df, X_test_scaled_df, y_test_df)
    else:
        evaluate_model(model, X_train_df, y_train_df, X_test_df, y_test_df)


Model: LogisticRegression
Confusion Matrix:
[[29810 11841     8     0     0    18   880]
 [10257 45243   618     2    46   306    28]
 [    0   741  5693   129     4   554     0]
 [    0     1   259   210     0    56     0]
 [   12  1889    69     0    15    10     0]
 [    0   820  1718    28     3   920     0]
 [ 1669    39     0     0     0     0  2307]]

Classification Report:
              precision    recall  f1-score   support

           1       0.71      0.70      0.71     42557
           2       0.75      0.80      0.77     56500
           3       0.68      0.80      0.74      7121
           4       0.57      0.40      0.47       526
           5       0.22      0.01      0.01      1995
           6       0.49      0.26      0.34      3489
           7       0.72      0.57      0.64      4015

    accuracy                           0.72    116203
   macro avg       0.59      0.51      0.53    116203
weighted avg       0.71      0.72      0.71    116203

Accuracy: 0.724576

In [1]:
cols = open("./datasets/kdd/kddcup.names",'r').read()
cols = [c[:c.index(':')] for c in cols.split('\n')[1:-1]]
cols.append('attack')

# attacks = open("./datasets/kdd/training_attack_types",'r').read()
# attacks = {t[0] : t[1] for t in (t.split(' ') for t in attacks.split('\n')) if len(t) == 2}
# attacks['normal'] = 'normal'
# attacks

all_df = pd.read_csv("./datasets/kdd/corrected", names = cols)
all_df['attack'] = all_df['attack'].apply(lambda r:TARGET_DICT[r[:-1]])

print('Normal class: ', all_df[TARGET_COLUMN].mode())
print('Feature  names: ', data.feature_names)
all_df.head()

NameError: name 'pd' is not defined